# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一片宁静而开阔的自然景观。中间有一条木栈道向前延伸，两侧是郁郁葱葱的草地和灌木。背景是蓝天，点缀着几朵白云，阳光洒落在草地上，营造出温暖而舒适的氛围。整体感觉宁静、自然，适合散步和享受大自然的美景。', role='assistant', function_call=None, tool_calls=None, refusal=None))]


In [2]:
response.choices[0].message.content

'这幅图描绘了一片宁静而开阔的自然景观。中间有一条木栈道向前延伸，两侧是郁郁葱葱的草地和灌木。背景是蓝天，点缀着几朵白云，阳光洒落在草地上，营造出温暖而舒适的氛围。整体感觉宁静、自然，适合散步和享受大自然的美景。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图展示了两个对比的形象，一边是“16岁的我”，一边是“工作后的我”。左边的形象是一只肌肉发达的狗（通常被称为柴犬），配文表达了年轻时的自信和对未来的美好期待。右边的形象是同样的狗，但看起来更加疲惫，配文则反映了工作后的压力和生活的沮丧感。

整体上，这幅图通过夸张的对比，形象地表现了从年轻充满活力到成年后面临压力与普通生活的转变，引发观者对成长和生活挑战的共鸣。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [5]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图展示了1980年至2020年间四个国家的GDP比较，包括美国、中国、日本和德国。以下是主要内容：

1. **横轴**表示年份，从1980年到2020年。
2. **纵轴**表示GDP，以万亿美元为单位。
3. **不同颜色的线条**代表不同国家：
   - **蓝色**：美国（USA）
   - **红色**：中国（China）
   - **紫色**：日本（Japan）
   - **绿色**：德国（Germany）

### 观察点：
- **美国的GDP**在整个时间段内保持领先，且呈现出稳定的增长趋势。
- **中国的GDP**自1980年以来增长显著，尤其是2000年后，大幅度上升，逐渐缩小与美国的差距。
- **日本的GDP**在1990年代初期达到峰值后，增长缓慢，显示出经济停滞的状态。
- **德国的GDP**在1990年代后逐渐增长，但总体增速不如中国。

总体来看，这张图清晰展示了全球经济格局的变化，尤其是中国在过去几十年间的快速崛起。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [13]:
content = query_base64_image_description_v2("./images/handwriting_0.jpg")
print(content)

这张图的内容是关于“参数高效微调”（PEFT）的一些主要概念，特别是关于几种微调技术的具体公式和说明。以下是主要内容的解释：

1. **Prompt Tuning**:
   - 通过使用一个小模型和嵌入（Embedding）层来对输入数据进行处理。
   - 公式表示了如何利用输入的序列 \(X\) 来生成变换后的序列 \(X'\)，最后利用权重 \(W\) 计算输出 \(Y\)。

2. **Prefix Tuning**:
   - 这里使用了一个权重矩阵 \(W\)，结合输入的特征进行编码和解码。
   - 提供了如何构建特定模型的概念，类似于如何使用 Transformer 模型进行微调。

3. **LoRA（低秩适配）**:
   - 该部分探讨了通过低秩矩阵调整模型参数的方法。
   - 提供了如何将新的权重 \(\Delta W\) 引入原始权重 \(W\) 中，以达到更有效的模型训练。

4. **QLoRA（量化低秩适配）**:
   - 这是对 LoRA 方法的一种变体，涉及模型的量化。
   - 可能列出了两个不同版本的模型与其所需的存储空间要求。

整体来看，该笔记主要记录了在机器学习和深度学习领域中，针对模型微调的一些重要方法和公式。

None


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [8]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张笔记中的内容涉及到微调（Tuning）和参数高效微调的方法，具体包括以下几个部分：

1. **Prompt Tuning**：
   - 讨论了使用固定模型大小和少量参数进行的嵌入（Embedding）处理。
   - 公式中，\(X\) 表示输入数据，\(W\) 是权重矩阵，\(Y\) 是输出。

2. **Prefix Tuning**：
   - 介绍了如何通过权重结合（使用 \(W\) 和小的 \(W_p\)）来增强模型性能。
   - 公式表达了与Transformer编码器/解码器的关系。

3. **LoRA（低秩适应）**：
   - 提出了用低秩方法来调整模型，使其在处理输入 \(X\) 时，能够更高效地利用参数。
   - 公式展示了 \(Y\) 的计算方法以及如何通过引入额外的参数\(\Delta W\)来实现调整。

4. **QLoRA**：
   - 讨论了QLoRA方法的应用和不同版本模型的大小比较。

整体来看，这些内容主要围绕如何优化机器学习模型的参数，通过不同的微调策略来提高模型在特定任务中的表现。

![](./images/handwriting_1.jpg)

In [9]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图是一篇关于多模态指令微调（Instruction Fine-tuning）和模型调整（Model Tuning）方法的笔记。具体内容包括：

1. **I-F Transformers**：提到了一些基于变换器的模型和基准测试（Benchmark）。

2. **PEFT（Parameter Efficient Fine Tuning）**：列出了当前的最佳实践（SOTA）和方法，包括各种提示调整（Prompt Tuning）技术，如 Adapter、Prefix Tuning 和 P-Tuning。

3. **Prompt Tuning**：详细说明了不同年份的提示调整方法，以及它们的来源和应用。

4. **多模态和LLMs**：提到了一些新的方法，如 LLava 和 LoRA（Low-Rank Adaptation），以及与 PETC 相关的内容。

5. **Adapters 和 Prefix Tuning**：涉及如何通过调整不同类型的适配器和前缀来增强模型性能。

6. **最后一部分**：提到了一些具体的研究成果和方法，涉及不同的适配器类型和提示策略的比较及其效果。

整体来看，这是一份关于深度学习模型微调和优化方法的综述，特别关注如何提高模型的效率和性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [12]:
# 作业2
from openai import OpenAI
import base64
import requests
from IPython.display import display, Markdown

client = OpenAI()  # 初始化 OpenAI 客户端


def query_base64_image_description_v2(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return display(Markdown(content))
    else:
        return f"Error: {response.status_code}, {response.text}"